In [3]:
# Transferred that functions into separate file
# IN otder to keep the code clean

from misc import loadProperties, loadWEKA

In [4]:
props = loadProperties('submitActionClass.properties')
(data, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)

### Matching YT video titles with their location
Naive way looping through the title and description word by word, trying to match the location:

In [32]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

Attempt to adjust a pipeline for better location recognition:

In [192]:
nlp = spacy.load("en_core_web_sm")

locations = [x for x in props] # Array of known locations from .properties
location_patterns = list(nlp.pipe(locations))

matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
matcher.add("LOCATION", None, *location_patterns)

# Define the custom component
def location_component(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label 'LOCATION'
    # Overwrite the doc.ents with the matched spans
    doc.ents = [Span(doc, start, end, label="LOCATION") for match_id, start, end in matches]
    return doc

# Add the component to the pipeline after the 'ner' component
nlp.add_pipe(location_component, before='ner')

# Just an example of recognition
doc = nlp("Snorkeling in Thudufushi Thailand in Mexico")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Thudufushi', 'GPE'), ('Thailand', 'LOCATION'), ('Mexico', 'GPE')]


In [131]:
# Uses spacy to look up for location in a strings. Returns Array of matches
def nlpLocation(string):
    spacy_mathc = []
    matches = 0
    
    for ent in nlp(string).ents:
        if ent.label_ == "GPE" or ent.label_ == "LOCATION":
            spacy_mathc.append(ent.text)
            matches += 1

    return spacy_mathc

In [193]:
def matchLocation(data, props, description=False, output=False):
    entries_processed = 0
    matched = 0
    spacy_found = 0
    
    for item in data:
        # Working with YT videos only
        if item[1]:
            naive_match = [] # [[matches_in_title], [mathces_in_descr]]
            spacy_match = [] # [[identified_in_title], [identified_in_descr]]
            
            # Going through title word by word
            title_splitted = item[2].split()
            title_match = []
            for word in title_splitted:
                try:
                    if (props[word.lower()]):
                        title_match.append(word)
                except:
                    pass
                
            naive_match.append(title_match)
            
            # Use spacy if location was not matched in a title
            spacy_match.append(nlpLocation(item[2].replace("'","")))
            
            # Parse description as well
            if description:
                # Going through description word by word
                descr_splitted = item[4].split()
                descr_match = []
                for word in descr_splitted:
                    try:
                        if (props[word.lower()]):
                            descr_match.append(word)
                    except:
                        pass
                    
                naive_match.append(descr_match)
                    
                # Use spacy if location was not matched in a description
                spacy_match.append(nlpLocation(item[4].replace("'","")))
    
            # Counting statistics
            entries_processed += 1
            matched += 1 if sum(len(x) for x in naive_match)>0 else 0
            spacy_found += 1 if sum(len(x) for x in spacy_match)>0 else 0
            
            if output:
                print(item)
                print("Naive match", naive_match)
                print("Spacy match", spacy_match)
                print()

    print("Processed ", entries_processed, " videos")
    print("Naively matched ", matched, " locations")
    print("Identified by spacy ", spacy_found, " locations")
    
matchLocation(data, props, description=True, output=True) # Pass data[:11800] to see a small bunch of results

['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriWhale sharkSqualo balenaSharksSqualiMurenaMorayStingrayTrigoneAquila di mareRays fishPesce chirurgoSurgeonfishLion fishpesce leonepesce scorpioneTartarugheturtlesmantasnorkelinggopro hero 7 blackgopro hero 4 black', 'In this video you can see the Whale shark Sharks Manta Stingray Surgeonfish Lion fish Turtles Rays fish and much moreFilmed with GoPro Hero 7 Black and GoPro Hero 4 BlackIn questo video potrete vedere lo squalo balena diversi squali tartarughe pesci chirurgo mante trigone aquile di mare e molto altroFilmato con GoPro Hero 7 Black e GoPro hero 4 Black', 'Ari Atol Maldive', 'Maldives']
Naive match [[], []]
Spacy match [['Thudufushi'], ['Sharks Manta Stingray']]

['53fd808d-9759-47fc-ac18-63a4e3715afc', 'https://www.youtube.com/watch?v=sziYTviQa74', 'Requin baleine  Whale shark', '', '', '

['6d389bf8-980b-4dfc-88b0-d607812b41bd', 'https://www.youtube.com/watch?v=pWlzd38QiZE', 'whale shark snorkel Isla Mujeres Mexico', '', '', 'mujeres', '2c']
Naive match [['Mujeres'], []]
Spacy match [['Mujeres'], []]

['941364ae-a8bf-40d5-ad80-cf5b2ca609ee', 'https://www.youtube.com/watch?v=p1wgd-xM2VQ', 'Maldives Swimming in Maldives with Whale Shark', '', '', 'maldives', 'Maldives']
Naive match [['Maldives', 'Maldives'], []]
Spacy match [['Maldives', 'Maldives'], []]

['f8fa15da-6e8e-4c13-b3b6-6bb41dceaafe', 'https://www.youtube.com/watch?v=T7JNdio47dU', '2018 SharkyDJ3 Mission english', '', '', 'djibouti', 'Djibouti']
Naive match [[], []]
Spacy match [[], []]

['f17c767f-163a-49b2-a626-73e0267911e6', 'https://www.youtube.com/watch?v=fHJ-RmmlMWU', 'WhalesharkBEST', '', '', '', 'null']
Naive match [[], []]
Spacy match [[], []]

['2c002a69-33e7-457a-ab3c-5f06ad59524e', 'https://www.youtube.com/watch?v=4X0Gz3OjzNU', 'Chumphon Thailand and Whale shark', '', '', 'thailand chumphon', 'Thail

['29a38cdd-145b-4b10-9a66-f860f5bf4e25', 'https://www.youtube.com/watch?v=fWMmxmSeeUc', 'The biggest fish in the sea the Whaleshark  Big Blue Diving Koh Tao Thailand', 'whalesharkwhale sharkkoh tao scuba divingkoh tao divingkoh tao diving schoolsscuba diving lessonsdiving schoolscuba diving certificationscuba diving coursediving lessonsscuba shopdiving certificationdiving in thailandpadi coursesscuba diving licensediving licensedive shopkoh taoko taoscuba divingscubadivingbig blue divingbig bluebigbluekoh phangankoh thaibest diving in thailandlearn to divepadi', 'The whale shark Rhincodon typus is a slowmoving filterfeeding carpet shark and the largest known extant fish species The largest confirmed individual had a length of 1265 m 415 ft and a weight of about 215 t 47000 lb This whale shark footage was from Southwest Pinnacle Koh Tao ThailandCheck out our website  social mediawwwbigbluedivingcomInstagram  Facebook  Facebook  ', 'thailand koh', 'Thailand']
Naive match [['Koh', 'Thaila

['eb6def57-1882-41bf-8db1-d8229e29425c', 'https://www.youtube.com/watch?v=pRT37UhrL2k', 'South Cebu', '', '', 'cebu sumilon', '6a2']
Naive match [['Cebu'], []]
Spacy match [['Cebu'], []]

['6e004545-ab52-479e-a6a5-fe42ba65c2cd', 'https://www.youtube.com/watch?v=bJjtYF45T24', 'whale shark 1', '', 'I swam with whale sharks in Cancun WOW how beautiful are these creatures ', 'cancun', '2c']
Naive match [[], ['Cancun']]
Spacy match [[], ['Cancun']]

['9d54c3e5-0fae-4198-a8ca-33108525b685', 'https://www.youtube.com/watch?v=cJLJ_umb3N8', 'I freak out swimming with a whale shark', '', '', '', '2c']
Naive match [[], []]
Spacy match [[], []]

['a17bfca0-38bb-4a00-910f-209aa7204097', 'https://www.youtube.com/watch?v=SzNv35d7ZXs', 'Swimming with the whale sharks in Cebu', '', '', 'cebu', '6a2']
Naive match [['Cebu'], []]
Spacy match [['Cebu'], []]

['307c25a5-35fd-44f2-b425-0317b0dcbba9', 'https://www.youtube.com/watch?v=JQEaC5UzKu0', 'Whale Shark Snorkel 01', 'iMovie', 'This video is about My Mov

['503e3a32-a0db-489a-8d07-90cd52a7d1c2', 'https://www.youtube.com/watch?v=ezlRg-kWCFc', 'My close encounter with the whale shark', '', '', '', 'null']
Naive match [[], []]
Spacy match [[], []]

['7d380fdb-96f4-4531-a2f9-1c8ebb1b6924', 'https://www.youtube.com/watch?v=L8jeuOcUkkg', 'Swimming with Whale Shark  Indian Ocean  Maldives', 'atolloarisnorkelingsqualobalenamaldivemaldivestraveltravellermareoceanoindianosqualobalenanuotareswimmingwhaleshark', 'Swimming with Whale Shark  Indian Ocean  Maldives    Nuotare con squalo Balena  oceano indiano  Maldive  Enjoy ', 'maldives', 'Maldives']
Naive match [['Maldives'], ['Maldives']]
Spacy match [['Maldives'], ['Maldives']]

['7dd20787-2f21-4fdb-ac5d-de4575c94e32', 'https://www.youtube.com/watch?v=R_1AHVO1F7A', 'Dhigurah Island A Beach  Diving Paradise In The Maldives', 'maldivesasiatraveltravelbloggertraveltipsdjidronegoproparadisebeachunderwaterwhale sharkdivingpadi', 'A journey to Dhigurah island in the Maldives on a whim turned out to be e

...

['3c1b0016-0e9d-4276-9420-be54be2ea2b0', 'https://www.youtube.com/watch?v=poSlUKxPXJM', 'Whale Shark Adventure 2017', 'whale shark tourwhalesharkscancunbestsnorkelbest snorkelswim with mantasswim with whale sharks', 'What an awesome day out with the whale sharks and manta raysYou want to know more or join a tripwwwencounterscommx', 'cancun', '2c']
Naive match [[], []]
Spacy match [[], []]

['52f10901-daa7-4467-be02-3e56fa587079', 'https://www.youtube.com/watch?v=poSlUKxPXJM', 'Whale Shark Adventure 2017', 'whale shark tourwhalesharkscancunbestsnorkelbest snorkelswim with mantasswim with whale sharks', 'What an awesome day out with the whale sharks and manta raysYou want to know more or join a tripwwwencounterscommx', 'cancun', '2c']
Naive match [[], []]
Spacy match [[], []]

['d56e9420-e275-4491-a35c-19397c086c0e', 'https://www.youtube.com/watch?v=poSlUKxPXJM', 'Whale Shark Adventure 2017', 'whale shark tourwhalesharkscancunbestsnorkelbest snorkelswim with mantasswim with whale sharks'

['b7084f58-5a72-488e-8123-eb210ac37efb', 'https://www.youtube.com/watch?v=EkfFXU3yC3k', 'Can you get eaten by a Whale Shark Swimming with Whale Sharks in Oslob', '', '', 'oslob philippines', '6a2']
Naive match [['Oslob'], []]
Spacy match [['Oslob'], []]

['8230779c-b247-488c-8621-aed6428926a9', 'https://www.youtube.com/watch?v=-Daz32yXi0k', 'Mala Boutique Dhangethi  Holiday Destination Maldives', '', '', 'destin maldives', 'Maldives']
Naive match [['Maldives'], []]
Spacy match [['Maldives'], []]

['86f8a24a-1f75-48ea-b160-72a15850c8ab', 'https://www.youtube.com/watch?v=J45jThNCyGg', 'Whale Shark at Sail Rock 10 February 2018 Underwater video', '', '', 'sail rock', 'Thailand']
Naive match [[], []]
Spacy match [[], []]

['b524f17b-8b6b-4cc5-89d6-197a5ab8bdba', 'https://www.youtube.com/watch?v=mgeRGK9JYSo', 'Galpagos Islands', '', '', 'galapagos', '7a']
Naive match [[], []]
Spacy match [[], []]

['30030c28-c6e5-4332-9a04-7532d754dc3d', 'https://www.youtube.com/watch?v=rXm0qLQ-DG4', 'Divin

['e46e5900-735d-4910-a185-844f1751ed51', 'https://www.youtube.com/watch?v=2Q_4z00sOhY', 'Big Fish', '', '', 'maldives', 'Maldives']
Naive match [[], []]
Spacy match [[], []]

['44756568-5a97-4507-a516-7f3511b7838f', 'https://www.youtube.com/watch?v=_fiCN8u2z84', 'Whaleshark watching 2018 oslob cebu', '', '', 'oslob cebu', '6a2']
Naive match [['oslob', 'cebu'], []]
Spacy match [['oslob', 'cebu'], []]

['c18c6689-3a07-40b5-ac08-82175758bef8', 'https://www.youtube.com/watch?v=BsQAPoddXUk', 'Richard Branson   WildAid Whale Sharks   2011', '', '', 'mujeres', '2c']
Naive match [[], []]
Spacy match [[], []]

['5e776f95-f4e8-4a8b-aa98-809cbb9ac035', 'https://www.youtube.com/watch?v=8ETA79SCtLg', 'Atmosphere Resort  Day Trip to Oslob Cebu  Whale Sharks  Philippines', '', '', 'oslob philippines cebu', '6a2']
Naive match [['Oslob', 'Cebu', 'Philippines'], []]
Spacy match [['Oslob', 'Cebu', 'Philippines'], []]

['7bdfa12b-fc0f-44b1-a484-decc099a6516', 'https://www.youtube.com/watch?v=NeVhq5uoJFY',

['3d1a6277-0c40-4e31-9799-35c7a655dc31', 'https://www.youtube.com/watch?v=MgFdgwEJQDE', 'Whale Shark experience for the first time', '', '', 'Bentota  Sri Lanka', 'null']
Naive match [[], []]
Spacy match [[], []]

['35a12538-527d-460d-a0c3-f8f8f71b5ac0', 'https://www.youtube.com/watch?v=MwxV-sjqq9M', 'Its A Divers World', '', '', 'maldives', 'Maldives']
Naive match [[], []]
Spacy match [[], []]

['754216ba-4a56-4894-99b9-ef6fdb4b1421', 'https://www.youtube.com/watch?v=-nrlgU0XAeI', 'Cuba Whale Shark Garden Of the Queen', '', '', 'Cuba Gardens of the Queen', '2f']
Naive match [[], []]
Spacy match [['Cuba'], []]

['8ca5a3eb-4d12-40aa-a2ea-e8dec1e042da', 'https://www.youtube.com/watch?v=-nrlgU0XAeI', 'Cuba Whale Shark Garden Of the Queen', '', '', 'Cuba Gardens of the Queen', '2f']
Naive match [[], []]
Spacy match [['Cuba'], []]

['67e26115-b1dc-40b8-a452-922411c7b246', 'https://www.youtube.com/watch?v=-nrlgU0XAeI', 'Cuba Whale Shark Garden Of the Queen', '', '', 'Cuba Gardens of the Quee

['9b6252fd-0a46-4142-b4ac-795f153c357f', 'https://www.youtube.com/watch?v=0AMzgY231XQ', 'INCREIBLE Mira como este tiburon ballena ataca a un buzo 2017', 'video de animales 2017video viral de animalesvideo gracioso de animalesvideo de risa 2017video de perros graciososvideo de animales salvajesvideo de gatos graciososataque de ballenaataque de tiburontiburon atacaterror en el mar', 'Gua de como entrenar a tu perro para trepar rboles subir colinas nadar en la playa y ms ac Historia de los perros Terrier Lista de reproduccin de un Jack Russell trepando rboles subiendo colinas nadando en la playa y ms ac JACK RUSSELL TREPANDO RBOLES Quieres conocer mejor a tu Terrier Revisa este blog TAGSIGNORARperro terrier ingles perro terrier escoces perro terrier ladrando raza de perro terrier perro bull terrier perro jack russell terrier perro bull terrier vs toro perro bull terrier fue ahorcado en un poste en pitalito huila perro fox terrier perro bull terrier ingles perro terrier perro bull terrier 